In [3]:
from astroquery.gaia import Gaia
from astropy.table import Table, vstack
import numpy as np

# We'll impose a magnitude limit to keep the dataset somewhat manageable.
# For example, G < 18 or G < 19.5. (Adjust as needed.)
g_max = 20.0

# If you try to do it in one single shot:
#   SELECT * 
#   FROM gaiaedr3.gaia_source
#   WHERE dec BETWEEN -2 AND 2
#   AND phot_g_mean_mag < g_max
# it might be extremely large and could time out. 
# Therefore, we do a loop over RA in small increments (e.g. 30° each)
# and merge the results.p

deg_step = 1
dec_min, dec_max = -2.0, 2.0

all_results = None

for ra_min in range(0, 360, deg_step):
    ra_max = ra_min + deg_step

    # Build your ADQL query
    query = f"""
    SELECT
      source_id,
      ra,
      dec,
      phot_g_mean_mag
    FROM gaiaedr3.gaia_source
    WHERE ra BETWEEN {ra_min} AND {ra_max}
      AND dec BETWEEN {dec_min} AND {dec_max}
      AND phot_g_mean_mag < {g_max}
    """
    print(f"Querying RA=[{ra_min}, {ra_max}), Dec=[{dec_min}, {dec_max}], G<{g_max}")
    
    # Run query
    job = Gaia.launch_job_async(query)
    result_table = job.get_results()
    print(f"  -> Returned {len(result_table)} rows")
    
    if all_results is None:
        all_results = result_table
    else:
        all_results = vstack([all_results, result_table])

# Optionally remove duplicates by source_id if they appear in overlapping edges
unique_ids, idx = np.unique(all_results['source_id'], return_index=True)
all_results = all_results[idx]

print(f"Final total rows: {len(all_results)}")

# Save to disk
out_file = "celestial_equator_band.fits"
all_results.write(out_file, overwrite=True)
print(f"Saved file: {out_file}")

Querying RA=[0, 1), Dec=[-2.0, 2.0], G<20.0
INFO: Query finished. [astroquery.utils.tap.core]
  -> Returned 7765 rows
Querying RA=[1, 2), Dec=[-2.0, 2.0], G<20.0
INFO: Query finished. [astroquery.utils.tap.core]
  -> Returned 7568 rows
Querying RA=[2, 3), Dec=[-2.0, 2.0], G<20.0
INFO: Query finished. [astroquery.utils.tap.core]
  -> Returned 7442 rows
Querying RA=[3, 4), Dec=[-2.0, 2.0], G<20.0
INFO: Query finished. [astroquery.utils.tap.core]
  -> Returned 7337 rows
Querying RA=[4, 5), Dec=[-2.0, 2.0], G<20.0
INFO: Query finished. [astroquery.utils.tap.core]
  -> Returned 7331 rows
Querying RA=[5, 6), Dec=[-2.0, 2.0], G<20.0
INFO: Query finished. [astroquery.utils.tap.core]
  -> Returned 7328 rows
Querying RA=[6, 7), Dec=[-2.0, 2.0], G<20.0
INFO: Query finished. [astroquery.utils.tap.core]
  -> Returned 7159 rows
Querying RA=[7, 8), Dec=[-2.0, 2.0], G<20.0
INFO: Query finished. [astroquery.utils.tap.core]
  -> Returned 6991 rows
Querying RA=[8, 9), Dec=[-2.0, 2.0], G<20.0
INFO: Query 